In [ ]:
# Install required packages
!pip install -qU langchain==0.3.12 langchain-chroma langchain-community pypdf langchain-openai wikipedia langgraph

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Environment setup
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Agent To Estimate VAPI Cost

In [ ]:
from langchain_core.tools import tool

@tool
def token_calculate_vapi(text: str) -> dict:
    """
    Estimates speech duration and usage for Vapi based on input text.
    Assumes ~13.3 characters per second of spoken output.
    """

    num_characters = len(text)
    chars_per_second = 13.3
    estimated_seconds = round(num_characters / chars_per_second, 2)
    estimated_minutes = round(estimated_seconds / 60, 2)

    return {
        "character_count": num_characters,
        "estimated_duration_seconds": estimated_seconds,
        "estimated_duration_minutes": estimated_minutes,
        "note": (
            "Estimated based on ~13.3 characters per second of speech. "
            "Useful for Vapi or other voice AI platforms that bill by duration."
        )
    }


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

key = userdata.get('OPENAI_API_KEY')
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

vapi_assistant = create_react_agent(
    model=model,
    tools=[token_calculate_vapi],
    prompt="""
You are a voice assistant that helps users estimate how long their text will take to be converted into speech using Vapi or similar voice AI platforms.

Instructions:
- Analyze the input text provided.
- Use the `token_calculate_vapi` tool to calculate:
  - Total character count.
  - Estimated speech duration in seconds and minutes.

Return your result strictly in the following JSON format:

{
  "character_count": ...,
  "estimated_duration_seconds": ...,
  "estimated_duration_minutes": ...,
  "note": "Estimated based on ~13.3 characters per second of speech. Useful for Vapi or other voice AI platforms that bill by duration."
}

Always use the `token_calculate_vapi` tool. Do not guess or create values manually.
""",
    name="vapi_assistant"
)


In [ ]:
text="Welcome to this 10-minute guided meditation. Begin by finding a comfortable seated or lying position. Gently close your eyes. Let your hands rest softly on your lap or by your sides. Take a deep breath in… and exhale slowly. Feel the weight of your body being supported by the ground beneath you. Now, take another deep breath in… and let it go. With each breath, allow your body to relax a little more. Soften your shoulders. Unclench your jaw. Relax the muscles around your eyes. Let go of any tension you may be holding. Bring your attention to your breath. Simply notice the inhale… and the exhale. Don’t try to change it. Just observe it as it flows naturally in and out. If your mind starts to wander, that’s okay. Gently bring it back to your breath. Now, take a moment to scan your body from head to toe. Notice any areas of tightness, and invite those areas to soften. Relax your forehead, your neck, your chest, your belly. Let everything melt and loosen with each breath. Let your thoughts come and go like clouds in the sky. No need to follow them or push them away. Just return to the rhythm of your breath—anchoring yourself in the present moment. You are safe. You are calm. You are grounded. Inhale deeply through your nose… and exhale gently through your mouth. With each breath, feel a sense of peace expanding within you. Now, bring to mind a feeling of gratitude. It could be for a person, a place, or even just the gift of this moment. Let that gratitude warm your heart. Take one more deep breath in… and sigh it out. As we near the end of this meditation, slowly bring your awareness back to your surroundings. Wiggle your fingers and toes. Gently roll your shoulders. When you're ready, open your eyes. Take a moment to notice how you feel. Carry this calm and clarity with you into the rest of your day. Thank you for taking this time for yourself."

In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
vapi_response = vapi_assistant.invoke({
    "messages": [
        HumanMessage(content=text)
    ]
})
vapi_response["messages"]

[HumanMessage(content="Welcome to this 10-minute guided meditation. Begin by finding a comfortable seated or lying position. Gently close your eyes. Let your hands rest softly on your lap or by your sides. Take a deep breath in… and exhale slowly. Feel the weight of your body being supported by the ground beneath you. Now, take another deep breath in… and let it go. With each breath, allow your body to relax a little more. Soften your shoulders. Unclench your jaw. Relax the muscles around your eyes. Let go of any tension you may be holding. Bring your attention to your breath. Simply notice the inhale… and the exhale. Don’t try to change it. Just observe it as it flows naturally in and out. If your mind starts to wander, that’s okay. Gently bring it back to your breath. Now, take a moment to scan your body from head to toe. Notice any areas of tightness, and invite those areas to soften. Relax your forehead, your neck, your chest, your belly. Let everything melt and loosen with each br

In [ ]:
vapi_response["messages"][-1].content

'{\n  "character_count": 1873,\n  "estimated_duration_seconds": 140.83,\n  "estimated_duration_minutes": 2.35,\n  "note": "Estimated based on ~13.3 characters per second of speech. Useful for Vapi or other voice AI platforms that bill by duration."\n}'